In [1]:
import pandas as pd
import mibian as mb
import yfinance as yf
from datetime import datetime

/Users/jteruya/.pyenv/versions/3.8.0/lib/python3.8/site-packages/pandas/compat/__init__.py:117: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


## Black Scholes Assumptions

In [2]:
## Assume volatility is 20
vol = 20

## Assume free interest rate is 3%
interest_rate = 3

## Yahoo Finance Ticker Object

In [3]:
stock_ticker = "CCL"
ticker = yf.Ticker(stock_ticker)

## Input DataFrame Helper Function

In [4]:
# Helper Functions for Constant Fields
def _constant_fields(df, expiration_date, interest_rate):
    df['underlying_price'] = ticker.history(period="0d").iloc[0]['Close']
    df['interest_rate'] = interest_rate
    df['days_to_expiration'] = (datetime.strptime(expiration_date, '%Y-%m-%d') - datetime.now()).days + 1

# Generate Mibian Input DF for Call Chains
def call_chain_input(ticker, expiration_date, interest_rate):
    df = pd.DataFrame(ticker.option_chain(expiration_date).calls[['strike']])
    _constant_fields(df, expiration_date, interest_rate)
    return df[['underlying_price', 'strike', 'interest_rate', 'days_to_expiration']]

# Generate Mibian Input DF for Put Chains
def put_chain_input(ticker, expiration_date, interest_rate):
    df = pd.DataFrame(ticker.option_chain(expiration_date).puts[['strike']])
    _constant_fields(df, expiration_date, interest_rate)
    return df[['underlying_price', 'strike', 'interest_rate', 'days_to_expiration']]

## Generate Black Scholes Input DataFrame Across Multiple Options Expirations
This steps takes a little time...

In [5]:
# Calls DF
call_input_df = pd.DataFrame()
# Puts DF
put_input_df = pd.DataFrame()

for expiration_date in ticker.options:
    call_input_df = call_input_df.append(call_chain_input(ticker=ticker, expiration_date=expiration_date, interest_rate=interest_rate), ignore_index=True)
    put_input_df = put_input_df.append(put_chain_input(ticker=ticker, expiration_date=expiration_date, interest_rate=interest_rate), ignore_index=True)

## Create Deep Copy of Input DataFrame
The previous step takes a while so this allows for rerunnability without having to re-run the previous step.

In [6]:
call_df = call_input_df.copy(deep=True)
put_df = put_input_df.copy(deep=True)

## Black Scholes Helper Functions

In [7]:
def black_scholes_vol(row, vol):
    return mb.BS(row.tolist(), volatility=vol)

def black_scholes_call_output(mb_object):
    return mb_object.callPrice, mb_object.callDelta, mb_object.callDelta2, mb_object.callTheta, mb_object.callRho, mb_object.vega, mb_object.gamma

def black_scholes_put_output(mb_object):
    return mb_object.putPrice, mb_object.putDelta, mb_object.putDelta2, mb_object.putTheta, mb_object.putRho, mb_object.vega, mb_object.gamma

## Generate Black Scholes Objects and Values

In [8]:
call_df['black_scholes'] = call_df.apply(lambda x: black_scholes_vol(x, vol), axis=1)
call_df['ticker'] = stock_ticker
call_df['black_scholes_call_price'], call_df['black_scholes_call_delta'], call_df['black_scholes_call_dual_delta'], call_df['black_scholes_call_theta'], call_df['black_scholes_call_rho'], call_df['black_scholes_vega'], call_df['black_scholes_gamma'] = \
    zip(*call_df['black_scholes'].map(lambda x: black_scholes_call_output(x)))

put_df['black_scholes'] = put_df.apply(lambda x: black_scholes_vol(x, vol), axis=1)
put_df['ticker'] = stock_ticker
put_df['black_scholes_put_price'], call_df['black_scholes_put_delta'], call_df['black_scholes_put_dual_delta'],put_df['black_scholes_put_theta'], put_df['black_scholes_put_rho'], put_df['black_scholes_vega'], put_df['black_scholes_gamma'] = \
    zip(*put_df['black_scholes'].map(lambda x: black_scholes_put_output(x)))

## Preview Call Output

In [9]:
call_df.head(10)

,underlying_price,strike,interest_rate,days_to_expiration,black_scholes,ticker,black_scholes_call_price,black_scholes_call_delta,black_scholes_call_dual_delta,black_scholes_call_theta,black_scholes_call_rho,black_scholes_vega,black_scholes_gamma,black_scholes_put_delta,black_scholes_put_dual_delta
0,17.23,1.0,3,3,<mibian.BS object at 0x12b693b80>,CCL,16.230247,1.0,-0.999753,-0.000082,0.000082,0.000000e+00,0.000000e+00,-0.000000e+00,0.000000e+00
1,17.23,2.0,3,3,<mibian.BS object at 0x12b693280>,CCL,15.230493,1.0,-0.999753,-0.000164,0.000164,0.000000e+00,0.000000e+00,-0.000000e+00,0.000000e+00
2,17.23,8.0,3,3,<mibian.BS object at 0x12b6938e0>,CCL,9.231972,1.0,-0.999753,-0.000657,0.000657,0.000000e+00,0.000000e+00,-0.000000e+00,0.000000e+00
3,17.23,9.0,3,3,<mibian.BS object at 0x12b693190>,CCL,8.232219,1.0,-0.999753,-0.000740,0.000740,7.524822e-282,1.541939e-279,-0.000000e+00,0.000000e+00
4,17.23,9.5,3,3,<mibian.BS object at 0x12b693f70>,CCL,7.732342,1.0,-0.999753,-0.000781,0.000781,2.281679e-237,4.675473e-235,-0.000000e+00,0.000000e+00
5,17.23,10.0,3,3,<mibian.BS object at 0x12b693610>,CCL,7.232465,1.0,-0.999753,-0.000822,0.000822,9.736300e-199,1.995102e-196,-0.000000e+00,0.000000e+00
6,17.23,10.5,3,3,<mibian.BS object at 0x12b5ec760>,CCL,6.732589,1.0,-0.999753,-0.000863,0.000863,3.222113e-165,6.602551e-163,-1.343068e-281,2.572529e-281
7,17.23,11.0,3,3,<mibian.BS object at 0x12b5ecd30>,CCL,6.232712,1.0,-0.999753,-0.000904,0.000904,3.465408e-136,7.101098e-134,-4.441385e-237,8.059709e-237
8,17.23,11.5,3,3,<mibian.BS object at 0x12b5ecf10>,CCL,5.732835,1.0,-0.999753,-0.000945,0.000945,4.071129e-111,8.342302e-109,-2.073376e-198,3.574581e-198
9,17.23,12.0,3,3,<mibian.BS object at 0x12b5eceb0>,CCL,5.232959,1.0,-0.999753,-0.000986,0.000986,1.467688e-89,3.007495e-87,-7.535257e-165,1.237318e-164


## Preview Put Output

In [10]:
put_df.head(10)

,underlying_price,strike,interest_rate,days_to_expiration,black_scholes,ticker,black_scholes_put_price,black_scholes_put_theta,black_scholes_put_rho,black_scholes_vega,black_scholes_gamma
0,17.23,4.0,3,3,<mibian.BS object at 0x12b693af0>,CCL,0.000000e+00,0.000000e+00,-0.000000e+00,0.000000e+00,0.000000e+00
1,17.23,5.0,3,3,<mibian.BS object at 0x12b6936d0>,CCL,0.000000e+00,0.000000e+00,-0.000000e+00,0.000000e+00,0.000000e+00
2,17.23,6.0,3,3,<mibian.BS object at 0x12b6e7460>,CCL,0.000000e+00,0.000000e+00,-0.000000e+00,0.000000e+00,0.000000e+00
3,17.23,7.0,3,3,<mibian.BS object at 0x12b6e7040>,CCL,0.000000e+00,0.000000e+00,-0.000000e+00,0.000000e+00,0.000000e+00
4,17.23,8.0,3,3,<mibian.BS object at 0x12b6e73d0>,CCL,0.000000e+00,0.000000e+00,-0.000000e+00,0.000000e+00,0.000000e+00
5,17.23,8.5,3,3,<mibian.BS object at 0x12b6e74c0>,CCL,0.000000e+00,0.000000e+00,-0.000000e+00,0.000000e+00,0.000000e+00
6,17.23,9.0,3,3,<mibian.BS object at 0x12b6e74f0>,CCL,1.169530e-283,-2.506371e-281,-1.902967e-284,7.524822e-282,1.541939e-279
7,17.23,9.5,3,3,<mibian.BS object at 0x12b6e7520>,CCL,4.217451e-239,-7.599303e-237,-6.293197e-240,2.281679e-237,4.675473e-235
8,17.23,10.0,3,3,<mibian.BS object at 0x12b6e7550>,CCL,2.153644e-200,-3.242495e-198,-2.938012e-201,9.736300e-199,1.995102e-196
9,17.23,10.5,3,3,<mibian.BS object at 0x12b6e7580>,CCL,8.593983e-167,-1.072970e-164,-1.067823e-167,3.222113e-165,6.602551e-163
